In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor, Slider
from matplotlib import colors

# Read the CSV file
df = pd.read_csv('finaleee.csv')

stars = df.to_dict('records')

# Constants for Kepler-22b
inclination = 89.764  # degrees
eccentricity = 0.72

# Create figure and axis with a black background
fig = plt.figure(figsize=(10, 6), facecolor='black')
ax = fig.add_subplot(111, polar=True)

# Calculate maximum distance for normalization, handle invalid distances
valid_distances = [s['distance (pc)'] for s in stars if pd.notnull(s['distance (pc)']) and isinstance(s['distance (pc)'], (int, float))]
if valid_distances:
    max_distance = np.max(valid_distances)
else:
    max_distance = 1  # Fallback if no valid distances are present

# Plot all stars with enhanced features
for i, star in enumerate(stars):
    # Use the correct column names
    l = star['l (deg)']
    b = star['b (deg)']
    distance = star['distance (pc)']  # Assuming distance is in the CSV file

    # Handle invalid l, b, or distance values
    if pd.isnull(l) or pd.isnull(b) or pd.isnull(distance):
        continue  # Skip stars with missing or invalid values
    if not isinstance(l, (int, float)) or not isinstance(b, (int, float)) or not isinstance(distance, (int, float)):
        continue  # Skip non-numeric values

    lon_rad = np.radians(l)
    lat_rad = np.radians(b)

    # Adjust transparency and color based on distance: exponential transparency decay
    transparency = np.exp(-distance / max_distance) if distance <= max_distance else 0  # Clamp transparency for valid distances
    transparency = np.clip(transparency, 0.05, 1)  # Ensure transparency is within a reasonable range

    # Color gradient based on distance: closer stars are warmer (yellowish), farther stars are cooler (bluish)
    color = colors.to_rgba('white')  # Default white
    color = (
        1 - (distance / max_distance),  # Red component decreases with distance
        1 - (distance / max_distance) ** 1.5,  # Green component fades with distance
        1,  # Blue stays constant (for cooler colors in distant stars)
        transparency  # Transparency based on distance
    )

    # Dynamic size scaling: closer stars appear larger
    size = 50 * (1 - (distance / max_distance))  # Size is inversely proportional to distance

    ax.scatter(lon_rad, np.abs(lat_rad), color=color, s=size, alpha=transparency)

# Customize the plot
ax.set_title('Star Chart from Kepler-22b Perspective', va='bottom', color='white')
ax.set_theta_direction(-1)  # Reverse direction of theta
ax.set_ylim(0, np.pi)  # Set limits for latitude
ax.set_rlim(0, 2)  # Increase the radius of the plot
ax.set_yticks([0, np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3, 5*np.pi/6, np.pi])  # Set y-axis tick locations
ax.set_yticklabels([])  # Remove y-axis tick labels

# Set the color of ticks and gridlines to white
ax.xaxis.set_tick_params(color='white')
ax.yaxis.set_tick_params(color='white')
ax.grid(color='white', linestyle='--')  # White gridlines with dashed lines

# Change latitude lines to white
for line in ax.get_yticklines():
    line.set_color('white')

# Change radial gridlines to white
ax.grid(color='white', linestyle='-', linewidth=1)  # White radial gridlines
ax.set_facecolor('black')  # Set the face color of the polar axes

# Set the font color of the tick labels to white
ax.tick_params(axis='both', colors='white')

# Add cursor for interactivity
cursor = Cursor(ax, useblit=True, color='yellow', linewidth=1)

# Zoom functionality with smoother interaction
def zoom(event):
    zoom_factor = 0.9 if event.key == 'up' else 1.1
    ax.set_ylim(ax.get_ylim()[0] * zoom_factor, ax.get_ylim()[1] * zoom_factor)  # Smooth zoom in/out
    plt.draw()

# Connect zoom function to key events
fig.canvas.mpl_connect('key_press_event', zoom)

# Add a slider widget to control zoom level
ax_zoom = plt.axes([0.25, 0.1, 0.65, 0.03])
slider_zoom = Slider(ax_zoom, 'Zoom', 0.01, 20, valinit=1, valstep=0.1)

# Define a function to update the plot when the zoom slider is changed
def update_zoom(val):
    zoom_level = slider_zoom.val
    ax.set_ylim(0, np.pi / zoom_level)
    plt.draw()

# Connect the update function to the zoom slider
slider_zoom.on_changed(update_zoom)

# Interactive hover function to display star information
def hover(event):
    if event.inaxes == ax:
        ax.set_title(f"Hovering: Theta={event.xdata:.2f}, Radius={event.ydata:.2f}", color='yellow')
    else:
        ax.set_title('Star Chart from Kepler-22b Perspective', color='white')
    plt.draw()

# Connect hover functionality
fig.canvas.mpl_connect('motion_notify_event', hover)

plt.show()
